In [2]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

In [10]:
from src.data.read_file import load_params, read_dataset
from sklearn.model_selection import train_test_split
import joblib
import pandas as pd


**[ DATA PREPARATION ]**

In [4]:
params_dir = "../config/config.yaml"
params = load_params(params_dir)
params

{'dataset_dir': 'data/raw/',
 'datetime_columns': ['tanggal'],
 'int32_columns': ['pm10', 'pm25', 'so2', 'co', 'o3', 'no2', 'max'],
 'label': 'categori',
 'label_categories': ['BAIK', 'SEDANG', 'TIDAK SEHAT'],
 'label_categories_new': ['BAIK', 'TIDAK BAIK'],
 'missing_value_co': 11,
 'missing_value_no2': 18,
 'missing_value_o3': 29,
 'missing_value_pm10': {'BAIK': 28, 'TIDAK BAIK': 55},
 'missing_value_pm25': {'BAIK': 38, 'TIDAK BAIK': 82},
 'missing_value_so2': 35,
 'object_columns': ['stasiun', 'critical', 'categori'],
 'predictors': ['stasiun', 'pm10', 'pm25', 'so2', 'co', 'o3', 'no2'],
 'range_co': [-1, 100],
 'range_no2': [-1, 100],
 'range_o3': [-1, 160],
 'range_pm10': [-1, 800],
 'range_pm25': [-1, 400],
 'range_so2': [-1, 500],
 'range_stasiun': ['DKI1 (Bunderan HI)',
  'DKI2 (Kelapa Gading)',
  'DKI3 (Jagakarsa)',
  'DKI4 (Lubang Buaya)',
  'DKI5 (Kebon Jeruk) Jakarta Barat']}

In [5]:
dataset = read_dataset('../' + params["dataset_dir"])
dataset.head(4)

100%|██████████| 12/12 [00:00<00:00, 222.50it/s]


,tanggal,stasiun,pm10,pm25,so2,co,o3,no2,max,critical,categori
0,2021-09-01,DKI1 (Bunderan HI),63,88,29,15,24,38,88,PM25,SEDANG
1,2021-09-02,DKI1 (Bunderan HI),60,83,29,11,30,28,83,PM25,SEDANG
2,2021-09-03,DKI1 (Bunderan HI),60,82,27,11,37,30,82,PM25,SEDANG
3,2021-09-04,DKI1 (Bunderan HI),58,77,26,10,31,28,77,PM25,SEDANG


In [6]:
dataset.describe()

,tanggal,stasiun,pm10,pm25,so2,co,o3,no2,max,critical,categori
count,1830,1830,1830,1768,1830,1830,1830,1830,1830,1813,1829
unique,335,5,122,191,121,54,120,86,186,6,4
top,2021-07-11,DKI1 (Bunderan HI),51,81,---,10,24,17,77,PM25,SEDANG
freq,10,366,72,41,114,171,67,90,45,1631,1305


In [7]:
joblib.dump(dataset, "../data/processed/dataset.pkl")

['../data/processed/dataset.pkl']

In [9]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1830 entries, 0 to 1829
Data columns (total 11 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   tanggal   1830 non-null   object
 1   stasiun   1830 non-null   object
 2   pm10      1830 non-null   object
 3   pm25      1768 non-null   object
 4   so2       1830 non-null   object
 5   co        1830 non-null   object
 6   o3        1830 non-null   object
 7   no2       1830 non-null   object
 8   max       1830 non-null   object
 9   critical  1813 non-null   object
 10  categori  1829 non-null   object
dtypes: object(11)
memory usage: 157.4+ KB


In [ ]:
dataset.tanggal = pd.to_datetime(dataset.tanggal)
